In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

# Preprocessing

In [5]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [6]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [7]:
pseudo = pd.read_csv("../input/in-python-tabular-denoising-residual-network/submission_v2.csv")[target]
test_pseudo = pd.concat([test, pseudo], axis=1)
all_pseudo = pd.concat([train, test_pseudo]).reset_index(drop=True)

# Optuna

In [8]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [9]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo[scale_features], y=all_pseudo[target]):
  """
  """
  param_space = {
                   'objective': 'regression',
              'metric':'rmse',
               #'device':'gpu',  # Use GPU acceleration
               # 'gpu_platform_id': 0,
                #    'gpu_device_id': 0,
               'n_estimators':N_ESTIMATORS,
              'learning_rate':trial.suggest_uniform('learning_rate', 1e-3, 3.7e-2),
               'subsample': trial.suggest_uniform('subsample', 0.658, 0.858),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.26, 0.46),
                'subsample_freq': trial.suggest_int('subsample_freq', 8, 10),
             #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 41.8, 53.8),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 31.4, 43.4),
            'min_child_weight':trial.suggest_uniform('min_child_weight', 184, 284),
                 'min_child_samples':trial.suggest_int('min_child_samples', 52, 76),
          'importance_type': 'gain'
             # 'num_leaves':trial.suggest_int('num_leaves', 10, 200),
              #'max_depth':trial.suggest_int('max_depth', 5, 50),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
              # 'n_jobs' : -1,
                #'max_bin':trial.suggest_int('max_bin', 300, 1000),
              #'cat_smooth':trial.suggest_int('cat_smooth', 5, 100),
              #'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 100)
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo[scale_features].iloc[trn_idx], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo[scale_features].iloc[oof_idx], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]

  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    lgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, lgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [10]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-26 07:26:33,901] A new study created in memory with name: no-name-80bdb0a9-b694-47f4-96a7-5f9700717b95


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.8063
[2000]	valid_0's rmse: 7.78609
[3000]	valid_0's rmse: 7.77512
[4000]	valid_0's rmse: 7.76901
[5000]	valid_0's rmse: 7.76484
[6000]	valid_0's rmse: 7.76212
[7000]	valid_0's rmse: 7.7602
[8000]	valid_0's rmse: 7.75892
[9000]	valid_0's rmse: 7.75797
[10000]	valid_0's rmse: 7.75719
Did not meet early stopping. Best iteration is:
[9927]	valid_0's rmse: 7.75718
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.80626
[2000]	valid_0's rmse: 7.78612
[3000]	valid_0's rmse: 7.77535
[4000]	valid_0's rmse: 7.76923
[5000]	valid_0's rmse: 7.76546
[6000]	valid_0's rmse: 7.763
[7000]	valid_0's rmse: 7.76131
[8000]	valid_0's rmse: 7.75986
[9000]	valid_0's rmse: 7.75853
[10000]	valid_0's rmse: 7.75755
Did not meet early stopping. Best iteration is:
[9991]	valid_0's rmse: 7.75754


[I 2021-08-26 07:38:43,872] Trial 0 finished with value: 7.757083842061544 and parameters: {'learning_rate': 0.0035512771785121597, 'subsample': 0.6762437106345379, 'colsample_bytree': 0.41395165351059676, 'subsample_freq': 9, 'reg_lambda': 47.4344602752043, 'reg_alpha': 32.609775659322544, 'min_child_weight': 256.67979686052695, 'min_child_samples': 64}. Best is trial 0 with value: 7.757083842061544.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76078
[2000]	valid_0's rmse: 7.75893
Early stopping, best iteration is:
[1811]	valid_0's rmse: 7.75811
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.7619
[2000]	valid_0's rmse: 7.75841
Early stopping, best iteration is:
[2158]	valid_0's rmse: 7.75763


[I 2021-08-26 07:40:59,853] Trial 1 finished with value: 7.75564091361693 and parameters: {'learning_rate': 0.030938760763348557, 'subsample': 0.8555570725689887, 'colsample_bytree': 0.3347722753422703, 'subsample_freq': 10, 'reg_lambda': 46.844649499072474, 'reg_alpha': 39.27755465958918, 'min_child_weight': 269.0218118597671, 'min_child_samples': 67}. Best is trial 1 with value: 7.75564091361693.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76342
[2000]	valid_0's rmse: 7.75899
Early stopping, best iteration is:
[2611]	valid_0's rmse: 7.75696
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76232
[2000]	valid_0's rmse: 7.75781
[3000]	valid_0's rmse: 7.75522
Early stopping, best iteration is:
[3026]	valid_0's rmse: 7.755


[I 2021-08-26 10:02:48,961] Trial 28 finished with value: 7.7536176587616685 and parameters: {'learning_rate': 0.026496618382746823, 'subsample': 0.8116742894611599, 'colsample_bytree': 0.3423729792932269, 'subsample_freq': 9, 'reg_lambda': 47.58098556620368, 'reg_alpha': 41.31461497333981, 'min_child_weight': 248.65579619463506, 'min_child_samples': 58}. Best is trial 8 with value: 7.752024327371435.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76475
[2000]	valid_0's rmse: 7.7594
[3000]	valid_0's rmse: 7.75593
Early stopping, best iteration is:
[3378]	valid_0's rmse: 7.75504
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76422
[2000]	valid_0's rmse: 7.75792
Early stopping, best iteration is:
[2724]	valid_0's rmse: 7.75581


[I 2021-08-26 10:06:26,600] Trial 29 finished with value: 7.753361165376201 and parameters: {'learning_rate': 0.020529979878830793, 'subsample': 0.697516721640106, 'colsample_bytree': 0.4213670872124987, 'subsample_freq': 9, 'reg_lambda': 47.46851632875074, 'reg_alpha': 40.06040835304397, 'min_child_weight': 240.48283279981067, 'min_child_samples': 64}. Best is trial 8 with value: 7.752024327371435.


Number of finished trials: 30
Best trial: {'learning_rate': 0.030576135597528146, 'subsample': 0.7073788145211187, 'colsample_bytree': 0.3902988247214292, 'subsample_freq': 9, 'reg_lambda': 48.496613691653494, 'reg_alpha': 41.598637848575336, 'min_child_weight': 256.2184024380349, 'min_child_samples': 53}


In [11]:
study.best_params

{'learning_rate': 0.030576135597528146,
 'subsample': 0.7073788145211187,
 'colsample_bytree': 0.3902988247214292,
 'subsample_freq': 9,
 'reg_lambda': 48.496613691653494,
 'reg_alpha': 41.598637848575336,
 'min_child_weight': 256.2184024380349,
 'min_child_samples': 53}

# Log

7.761521447852026 row-wise noise ver3

7.7627473037692365 random noise ver4

7.749394595681444 no noise ver6

7.7488157884604965 no noise ver8 (narrow space)

7.748628607085202 no noise ver10 (narrow space)

7.7483594115082495 no noise ver11 (narrow space)

7.748196039675071 no noise ver12 (narrow space) final

=====================================================
================= Change pseudo =====================

7.752717667960531 no noise ver15

7.752449992734185 no noise ver16 (narrow space)
